# 홈플러스 상품 데이터 크롤링 (2024년 5월 기준)
홈플러스 모바일몰(https://mfront.homeplus.co.kr/leaflet?gnbNo=207&homeType=MART)에서 Selenium을 활용해 신선식품 데이터를 크롤링합니다. 사이트 구조가 자주 바뀌므로, 크롤링이 안 될 경우 개발자도구로 실제 상품 블록의 CSS 선택자를 반드시 확인해야 합니다.

크롬드라이버는 https://googlechromelabs.github.io/chrome-for-testing/#stable 에서 다운로드했습니다.

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

def debug_leaflet_site():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    url = "https://mfront.homeplus.co.kr/leaflet?gnbNo=207&homeType=MART"
    driver.get(url)

    time.sleep(5)  # 로딩 대기

    # 전체 페이지 저장해서 확인
    html = driver.page_source
    with open("debug_leaflet.html", "w", encoding="utf-8") as f:
        f.write(html)
    
    print("페이지 저장 완료: debug_leaflet.html")
    driver.quit()

if __name__ == "__main__":
    debug_leaflet_site()


페이지 저장 완료: debug_leaflet.html


**전체 할인 품목 크롤링**

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time

def crawl_homeplus_all_discounts():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    driver.get("https://mfront.homeplus.co.kr/leaflet?gnbNo=207&homeType=MART")
    time.sleep(5)

    # 무한 스크롤: 끝까지 내려가며 데이터 로딩
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 스크롤 아래로 이동
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

        # 새로운 높이 계산 후 비교
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 더 이상 로딩된 데이터가 없음
        last_height = new_height

    # 데이터 추출
    products = []
    items = driver.find_elements(By.CLASS_NAME, "unitItem")
    for item in items:
        try:
            name = item.find_element(By.CLASS_NAME, "css-xktqki").text
            price = item.find_element(By.CLASS_NAME, "priceValue").text
            try:
                discount = item.find_element(By.CLASS_NAME, "discountRate").text
            except:
                discount = "없음"

            products.append({
                "상품명": name,
                "가격": price,
                "할인율": discount
            })
        except:
            continue

    driver.quit()
    return products

# 실행
if __name__ == "__main__":
    results = crawl_homeplus_all_discounts()
    print(f"총 {len(results)}개 상품을 찾았습니다.")
    for p in results:
        print(p)


총 1950개 상품을 찾았습니다.
{'상품명': '보먹돼 삼겹살 100G/돼지고기', '가격': '2,580', '할인율': '40%'}
{'상품명': '대추방울토마토 900G(박스)', '가격': '6,990', '할인율': '없음'}
{'상품명': '보먹돼 목심 100G/돼지고기(목살)', '가격': '2,580', '할인율': '40%'}
{'상품명': '다다기오이 5입(봉)', '가격': '3,740', '할인율': '없음'}
{'상품명': '12Brix 맛난이 사과 1.5KG(봉)', '가격': '12,490', '할인율': '없음'}
{'상품명': '햇 양파 1.7kg(망)', '가격': '3,740', '할인율': '없음'}
{'상품명': '미국산 냉동 만능 우삼겹 700G(팩)/소고기', '가격': '21,800', '할인율': '50%'}
{'상품명': '생생 한돈 고추장 불고기 800G', '가격': '14,990', '할인율': '40%'}
{'상품명': '[몽블랑제]슈크림 소보로 (8입)', '가격': '6,990', '할인율': '30%'}
{'상품명': '서울우유 2.3L', '가격': '5,990', '할인율': '없음'}
{'상품명': '양배추(국산) 1통', '가격': '4,240', '할인율': '없음'}
{'상품명': '휘라 노르웨이 횟감용 생연어 100G', '가격': '7,600', '할인율': '40%'}
{'상품명': '생생 한돈 간장 불고기 800G', '가격': '14,990', '할인율': '40%'}
{'상품명': '항공직송 캘리포니아 체리(미국) 450G(팩)', '가격': '16,990', '할인율': '29%'}
{'상품명': '[몽블랑제]The 좋은 버터롤 (8입)', '가격': '3,990', '할인율': '30%'}
{'상품명': '슈퍼푸드 블루베리(국산) 250G(팩)', '가격': '10,690', '할인율': '없음'}
{'상품명': '보먹돼 등갈비 100G/돼지고기', '가격': '3,020',

In [6]:
def crawl_homeplus_fresh_category():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    url = "https://mfront.homeplus.co.kr/leaflet?categoryId=20&sort=NEW"
    driver.get(url)
    time.sleep(5)

    # 2~3회만 스크롤 (신선식품 범위 내 로딩 충분)
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    products = []
    items = driver.find_elements(By.CLASS_NAME, "unitItem")
    for item in items:
        try:
            name = item.find_element(By.CLASS_NAME, "css-xktqki").text
            price = item.find_element(By.CLASS_NAME, "priceValue").text
            try:
                discount = item.find_element(By.CLASS_NAME, "discountRate").text
            except:
                discount = "없음"

            products.append({
                "상품명": name,
                "가격": price,
                "할인율": discount
            })
        except:
            continue

    driver.quit()
    return products

# 실행
if __name__ == "__main__":
    results = crawl_homeplus_fresh_category()
    print(f"신선식품 {len(results)}개 크롤링 완료")
    for p in results:
        print(p)


신선식품 91개 크롤링 완료
{'상품명': '11Brix 당도선별 수박 6KG이상(통)', '가격': '19,990', '할인율': '없음'}
{'상품명': '12Brix 맛난이 성주참외 1.5KG(봉)', '가격': '9,290', '할인율': '없음'}
{'상품명': '햇 양파 1.7kg(망)', '가격': '3,740', '할인율': '없음'}
{'상품명': '슈퍼푸드 블루베리(국산) 250G(팩)', '가격': '10,690', '할인율': '없음'}
{'상품명': '양배추(국산) 1통', '가격': '4,240', '할인율': '없음'}
{'상품명': '12Brix 맛난이 사과 1.5KG(봉)', '가격': '12,490', '할인율': '없음'}
{'상품명': '대추방울토마토 900G(박스)', '가격': '6,990', '할인율': '없음'}
{'상품명': '완숙토마토 2KG (박스)', '가격': '10,990', '할인율': '없음'}
{'상품명': '다다기오이 5입(봉)', '가격': '3,740', '할인율': '없음'}
{'상품명': '산지 그대로 당근 1.2KG(봉)', '가격': '3,740', '할인율': '없음'}
{'상품명': '남원 춘향愛인 복숭아 4입(팩)', '가격': '17,490', '할인율': '없음'}
{'상품명': '하우스 대석자두 400G(팩)', '가격': '10,990', '할인율': '없음'}
{'상품명': '깐마늘 300G(봉)', '가격': '4,990', '할인율': '없음'}
{'상품명': '국산 서리태 900G(봉)', '가격': '10,990', '할인율': '없음'}
{'상품명': '국산 찹쌀 2.5KG(봉)', '가격': '12,390', '할인율': '없음'}
{'상품명': '올해 첫 수확한 하우스밀감 700G(팩)', '가격': '14,590', '할인율': '없음'}
{'상품명': '농협안심한우 1+등급 등심 100G/소고기', '가격': '15,900', '할인율': '40%'}
{'상품

In [11]:
import pandas as pd

filename = "homeplus_discounts.csv"
df = pd.DataFrame(results)
df.to_csv(filename, index=False, encoding="utf-8-sig")  # 한글 인코딩을 위해 utf-8-sig 사용
print(f"✅ CSV 파일로 저장 완료: {filename}")


✅ CSV 파일로 저장 완료: homeplus_discounts.csv


In [13]:
def crawl_homeplus_fresh_with_links():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    url = "https://mfront.homeplus.co.kr/leaflet?categoryId=20&sort=NEW"
    driver.get(url)
    time.sleep(5)

    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    products = []
    items = driver.find_elements(By.CLASS_NAME, "unitItem")
    for item in items:
        try:
            name = item.find_element(By.CLASS_NAME, "css-xktqki").text
            price = item.find_element(By.CLASS_NAME, "priceValue").text
            try:
                discount = item.find_element(By.CLASS_NAME, "discountRate").text
            except:
                discount = "없음"
            link = item.find_element(By.CLASS_NAME, "productTitle").get_attribute("href")
            full_link = link if link.startswith("http") else "https://mfront.homeplus.co.kr" + link

            products.append({
                "상품명": name,
                "가격": price,
                "할인율": discount,
                "링크": full_link
            })
        except Exception as e:
            continue

    driver.quit()
    return products


# 실행
if __name__ == "__main__":
    results = crawl_homeplus_fresh_with_links()
    print(f"신선식품 {len(results)}개 상품 크롤링 완료")
    for p in results:
        print(p)


신선식품 91개 상품 크롤링 완료
{'상품명': '11Brix 당도선별 수박 6KG이상(통)', '가격': '19,990', '할인율': '없음', '링크': 'https://mfront.homeplus.co.kr/item?itemNo=121945057&storeType=HYPER'}
{'상품명': '12Brix 맛난이 성주참외 1.5KG(봉)', '가격': '9,290', '할인율': '없음', '링크': 'https://mfront.homeplus.co.kr/item?itemNo=105006636&storeType=HYPER'}
{'상품명': '햇 양파 1.7kg(망)', '가격': '3,740', '할인율': '없음', '링크': 'https://mfront.homeplus.co.kr/item?itemNo=121945149&storeType=HYPER'}
{'상품명': '슈퍼푸드 블루베리(국산) 250G(팩)', '가격': '10,690', '할인율': '없음', '링크': 'https://mfront.homeplus.co.kr/item?itemNo=061472989&storeType=HYPER'}
{'상품명': '양배추(국산) 1통', '가격': '4,240', '할인율': '없음', '링크': 'https://mfront.homeplus.co.kr/item?itemNo=000005495&storeType=HYPER'}
{'상품명': '12Brix 맛난이 사과 1.5KG(봉)', '가격': '12,490', '할인율': '없음', '링크': 'https://mfront.homeplus.co.kr/item?itemNo=068777382&storeType=HYPER'}
{'상품명': '대추방울토마토 900G(박스)', '가격': '6,990', '할인율': '없음', '링크': 'https://mfront.homeplus.co.kr/item?itemNo=070486337&storeType=HYPER'}
{'상품명': '완숙토마토 2KG (박스)', '가격'